In [20]:
# Automatically reload modules that have been changed
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchvision
from torchvision import transforms

from pathlib import Path


if 'image' not in vars():
        ## Laptop
        DATA_FOLDER = Path('/home/bahaa/Code/torch_nocs') / 'data'
        IMG_FOLDER = Path('train')
        ANNOTATION_FOLDER = Path('annotations_trainval2017/annotations/instances_train2017.json')

        ## Lab PC
        # DATA_FOLDER = Path('/home/baldeeb/Data/cocodataset/')
        # IMG_FOLDER = Path('val2017')
        # ANNOTATION_FOLDER = Path('annotations_trainval2017/annotations/instances_val2017.json')

        dataset = torchvision.datasets.CocoDetection(
                                    DATA_FOLDER/IMG_FOLDER
                                    ,DATA_FOLDER/ANNOTATION_FOLDER
                                    ,transform=transforms.PILToTensor())
        image, targets = dataset[0][0], dataset[0][1] 

loading annotations into memory...
Done (t=14.06s)
creating index...
index created!


In [3]:
from torchvision.models import ResNet50_Weights
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

from models.nocs import NOCS

backbone = resnet_fpn_backbone('resnet50', ResNet50_Weights.DEFAULT)
m = NOCS(backbone, 91)


m.eval()
# x = [torch.rand(3, 30, 40), torch.rand(3, 50, 40)]
x = [image.float() / 255.0]
predictions = m(x)


/home/bahaa/anaconda3/envs/nocs/lib/python3.8/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' and 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


: 

: 

In [18]:
print(predictions[0].keys())

dict_keys(['boxes', 'labels', 'scores', 'masks'])
